## Biblioteki

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Pobór danych 

In [3]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Długość ścieżki

In [4]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Zapis do pliku

In [19]:
def write_to_excel(file_path, sheet_name, df):
    try:
        # Load existing workbook or create a new one
        try:
            writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
            writer.book = load_workbook(file_path)
            if sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row
            else:
                startrow = 0
        except FileNotFoundError:
            writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='w')
            startrow = 0
        
        # Write DataFrame to the specified sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)
        
        # Save the workbook
        writer.close()
    except Exception as e:
        print(f"Error writing to Excel: {e}")

## Algorytm genetyczny (GA)

In [6]:
def crossover_order(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child = [None] * size
    child[start:end] = parent1[start:end]

    pointer = 0
    for gene in parent2:
        if gene not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = gene

    return child

def crossover_pmx(parent1, parent2):
    size = len(parent1)
    start, end = 2, 6
    child = [None] * size

    # Copy segment from parent1 to child
    child[start:end] = parent1[start:end]

    # Fill remaining positions from parent2
    for i in range(size):
        if child[i] is None:
            gene = parent2[i]
            while gene in child:
                gene = parent2[parent1.index(gene)]
            child[i] = gene

    return child

def selection_tournament(population, scores, k=3):
    selected = random.sample(range(len(population)), k)
    selected = sorted(selected, key=lambda x: scores[x])
    return population[selected[0]]

def selection_roulette(population, scores):
    total_score = sum(scores)
    
    # Calculate probabilities
    probabilities = [score / total_score for score in scores]

    # Calculate cumulative probabilities
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]
    
    # Pick a random number between 0 and 1
    pick = random.uniform(0, 1)

    # Select the individual based on the random number
    for i, cumulative_probability in enumerate(cumulative_probabilities):
        if pick <= cumulative_probability:
            return population[i]


def mutate_swap(individual):
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]

def mutate_inversion(individual):
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])

def mutate_insertion(individual):
    i, j = random.sample(range(len(individual)), 2)
    gene = individual.pop(i)
    individual.insert(j, gene)


def genetic_algorithm(distance_matrix, population_size=10, generations=5000, crossover_rate=0.6, mutation_rate=0.15, selection_method='tournament', crossover_method='order', mutation_method='swap'):
    start_time = time.time()

    n = len(distance_matrix)
    population = [random.sample(range(n), n) for _ in range(population_size)]
    best_solution = None
    best_cost = float('inf')
    best_time = 0

    for generation in range(1, generations + 1):
        scores = [calculate_path_cost(distance_matrix, individual) for individual in population]
        new_population = []

        for _ in range(population_size // 2):
            if selection_method == 'tournament':
                parent1 = selection_tournament(population, scores)
                parent2 = selection_tournament(population, scores)
            elif selection_method == 'roulette':
                parent1 = selection_roulette(population, scores)
                parent2 = selection_roulette(population, scores)

            if random.random() < crossover_rate:
                if crossover_method == 'order':
                    child1 = crossover_order(parent1, parent2)
                    child2 = crossover_order(parent2, parent1)
                elif crossover_method == 'pmx':
                    child1 = crossover_pmx(parent1, parent2)
                    child2 = crossover_pmx(parent2, parent1)
            else:
                child1, child2 = parent1[:], parent2[:]

            if random.random() < mutation_rate:
                if mutation_method == 'swap':
                    mutate_swap(child1)
                    mutate_swap(child2)
                elif mutation_method == 'inversion':
                    mutate_inversion(child1)
                    mutate_inversion(child2)
                elif mutation_method == 'insertion':
                    mutate_insertion(child1)
                    mutate_insertion(child2)

            new_population.extend([child1, child2])

        population = new_population

        for individual in population:
            cost = calculate_path_cost(distance_matrix, individual)
            if cost < best_cost:
                best_time = time.time() - start_time
                best_solution = individual
                best_cost = cost

        if (generation % 100 == 0) or (generation == generations):
            elapsed_time = time.time() - start_time  # Calculate elapsed time
            print(f"Iteration {generation}, Best Cost: {best_cost: .2f}, Elapsed Time: {elapsed_time:.2f} seconds")

    return best_solution, best_cost, best_time

### Generowanie wyników

Za podstawowe dane przyjmujemy:

* population_size = 10 

* generations = 5000

* crossover_rate = 0.6

* mutation_rate = 0.15

* selection_method = `tournament`
    
* crossover_method = `order`
    
* mutation_method = `swap`

In [7]:
# Dane_TSP_48
best_path_1, best_cost_1, best_time_1 = genetic_algorithm(data1)
# Dane_TSP_76
best_path_2, best_cost_2, best_time_2 = genetic_algorithm(data2)
# Dane_TSP_127
best_path_3, best_cost_3, best_time_3 = genetic_algorithm(data3)

Iteration 100, Best Cost:  25985.00, Elapsed Time: 0.04 seconds
Iteration 200, Best Cost:  22428.00, Elapsed Time: 0.08 seconds
Iteration 300, Best Cost:  21933.00, Elapsed Time: 0.11 seconds
Iteration 400, Best Cost:  21424.00, Elapsed Time: 0.14 seconds
Iteration 500, Best Cost:  21047.00, Elapsed Time: 0.17 seconds
Iteration 600, Best Cost:  20923.00, Elapsed Time: 0.21 seconds
Iteration 700, Best Cost:  20817.00, Elapsed Time: 0.24 seconds
Iteration 800, Best Cost:  20782.00, Elapsed Time: 0.28 seconds
Iteration 900, Best Cost:  20514.00, Elapsed Time: 0.31 seconds
Iteration 1000, Best Cost:  20514.00, Elapsed Time: 0.35 seconds
Iteration 1100, Best Cost:  20399.00, Elapsed Time: 0.39 seconds
Iteration 1200, Best Cost:  20118.00, Elapsed Time: 0.43 seconds
Iteration 1300, Best Cost:  18231.00, Elapsed Time: 0.47 seconds
Iteration 1400, Best Cost:  18231.00, Elapsed Time: 0.51 seconds
Iteration 1500, Best Cost:  17914.00, Elapsed Time: 0.55 seconds
Iteration 1600, Best Cost:  17670.

In [20]:
W1 =  {
    "Długość ścieżki": [best_cost_1, best_cost_2, best_cost_3],
    "Ścieżka": [best_path_1, best_path_2, best_path_3],
    "Czas": [best_time_1, best_time_2, best_time_3]
}

basic = pd.DataFrame(data = W1)
write_to_excel("GA.xlsx", "Basic", basic)

#### Badanie wpływu poszczególnych parametrów

Badanie wpływu parametru *population_size*

In [ ]:
def test_population_size(data, dataset_name, num_repeats=10):
    results = []
    population_sizes = [5, 10, 15, 20]
    for population_size in population_sizes:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, population_size=population_size)
            results.append({"PARAMETR": population_size, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_population_size_1 = test_population_size(data1, "DATA1")
df_population_size_2 = test_population_size(data2, "DATA2")
df_population_size_3 = test_population_size(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
population_size = pd.concat([df_population_size_1, df_population_size_2, df_population_size_3], ignore_index=True)
write_to_excel('GA.xlsx', 'population_size', population_size)

Badanie wpływu parametru *mutation_rate*

In [ ]:
def test_mutation_rate(data, dataset_name, num_repeats=10):
    results = []
    mutation_rates = [0.01, 0.05, 0.1, 0.2]
    for mutation_rate in mutation_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_rate=mutation_rate)
            results.append({"PARAMETR": mutation_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_rate_1 = test_mutation_rate(data1, "DATA1")
df_mutation_rate_2 = test_mutation_rate(data2, "DATA2")
df_mutation_rate_3 = test_mutation_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_rate = pd.concat([df_mutation_rate_1, df_mutation_rate_2, df_mutation_rate_3], ignore_index=True)
write_to_excel('GA.xlsx', 'mutation_rate', mutation_rate)

Badanie wpływu parametru *crossover_rate*

In [ ]:
def test_crossover_rate(data, dataset_name, num_repeats=10):
    results = []
    crossover_rates = [0.6, 0.7, 0.8, 0.9]
    for crossover_rate in crossover_rates:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_rate=crossover_rate)
            results.append({"PARAMETR": crossover_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_rate_1 = test_crossover_rate(data1, "DATA1")
df_crossover_rate_2 = test_crossover_rate(data2, "DATA2")
df_crossover_rate_3 = test_crossover_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_rate = pd.concat([df_crossover_rate_1, df_crossover_rate_2, df_crossover_rate_3], ignore_index=True)
write_to_excel('GA.xlsx', 'crossover_rate', crossover_rate)

Badanie wpływu parametru *generations*

In [ ]:
def test_generations(data, dataset_name, num_repeats=10):
    results = []
    generations_list = [5000, 7500, 10000, 15000]
    for generations in generations_list:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, generations=generations)
            results.append({"PARAMETR": generations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_generations_1 = test_generations(data1, "DATA1")
df_generations_2 = test_generations(data2, "DATA2")
df_generations_3 = test_generations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
generations = pd.concat([df_generations_1, df_generations_2, df_generations_3], ignore_index=True)
write_to_excel('GA.xlsx', 'generations', generations)

Badanie wpływu parametru *selection_method*

In [ ]:
def test_selection_method(data, dataset_name, num_repeats=10):
    results = []
    selection_methods = ['roulette', 'tournament']
    for selection_method in selection_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, selection_method=selection_method)
            results.append({"PARAMETR": selection_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_selection_method_1 = test_selection_method(data1, "DATA1")
df_selection_method_2 = test_selection_method(data2, "DATA2")
df_selection_method_3 = test_selection_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
selection_method = pd.concat([df_selection_method_1, df_selection_method_2, df_selection_method_3], ignore_index=True)
write_to_excel('GA.xlsx', 'selection_method', selection_method)

Badanie wpływu parametru *crossover_method*

In [ ]:
def test_crossover_method(data, dataset_name, num_repeats=10):
    results = []
    crossover_methods = ['order', 'pmx']
    for crossover_method in crossover_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, crossover_method=crossover_method)
            results.append({"PARAMETR": crossover_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_crossover_method_1 = test_crossover_method(data1, "DATA1")
df_crossover_method_2 = test_crossover_method(data2, "DATA2")
df_crossover_method_3 = test_crossover_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
crossover_method = pd.concat([df_crossover_method_1, df_crossover_method_2, df_crossover_method_3], ignore_index=True)
write_to_excel('GA.xlsx', 'crossover_method', crossover_method)

Badanie wpływu parametru *mutation_method*

In [ ]:
def test_mutation_method(data, dataset_name, num_repeats=10):
    results = []
    mutation_methods = ['insertion', 'swap', 'inversion']
    for mutation_method in mutation_methods:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = genetic_algorithm(data, mutation_method=mutation_method)
            results.append({"PARAMETR": mutation_method, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_mutation_method_1 = test_mutation_method(data1, "DATA1")
df_mutation_method_2 = test_mutation_method(data2, "DATA2")
df_mutation_method_3 = test_mutation_method(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
mutation_method = pd.concat([df_mutation_method_1, df_mutation_method_2, df_mutation_method_3], ignore_index=True)
write_to_excel('GA.xlsx', 'mutation_method', mutation_method)